In [1]:
# 从sklearn 调入所需要的包
# from sklearn import datasets
from sklearn.model_selection import train_test_split #数据分隔出训练集和验证集
import lightgbm as lgb
import numpy as np 
import pandas as pd
#导入精度和召回
from sklearn.metrics import precision_score, recall_score

In [2]:
pd.set_option('display.max_columns',None) #显示所有列

In [3]:
path='./'
train = pd.read_csv(path+'input/pfm_train.csv')
test = pd.read_csv(path+'input/pfm_test.csv')
print(train.info())
print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1100 entries, 0 to 1099
Data columns (total 31 columns):
Age                         1100 non-null int64
Attrition                   1100 non-null int64
BusinessTravel              1100 non-null object
Department                  1100 non-null object
DistanceFromHome            1100 non-null int64
Education                   1100 non-null int64
EducationField              1100 non-null object
EmployeeNumber              1100 non-null int64
EnvironmentSatisfaction     1100 non-null int64
Gender                      1100 non-null object
JobInvolvement              1100 non-null int64
JobLevel                    1100 non-null int64
JobRole                     1100 non-null object
JobSatisfaction             1100 non-null int64
MaritalStatus               1100 non-null object
MonthlyIncome               1100 non-null int64
NumCompaniesWorked          1100 non-null int64
Over18                      1100 non-null object
OverTime              

In [4]:
train.describe()#各个特征的分布

,Age,Attrition,DistanceFromHome,Education,EmployeeNumber,EnvironmentSatisfaction,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
count,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.0,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000
mean,36.999091,0.161818,9.427273,2.922727,1028.157273,2.725455,2.730909,2.054545,2.732727,6483.620909,2.683636,15.235455,3.152727,2.696364,80.0,0.788182,11.221818,2.807273,2.746364,7.011818,4.207273,2.226364,4.123636
std,9.037230,0.368451,8.196694,1.022242,598.915204,1.098053,0.706366,1.107805,1.109731,4715.293419,2.510017,3.628571,0.359888,1.095356,0.0,0.843347,7.825548,1.291514,0.701121,6.223093,3.618115,3.313830,3.597996
min,18.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1009.000000,0.000000,11.000000,3.000000,1.000000,80.0,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,30.000000,0.000000,2.000000,2.000000,504.250000,2.000000,2.000000,1.000000,2.000000,2924.500000,1.000000,12.000000,3.000000,2.000000,80.0,0.000000,6.000000,2.000000,2.000000,3.000000,2.000000,0.000000,2.000000
50%,36.000000,0.000000,7.000000,3.000000,1026.500000,3.000000,3.000000,2.000000,3.000000,4857.000000,2.000000,14.000000,3.000000,3.000000,80.0,1.000000,10.000000,3.000000,3.000000,5.000000,3.000000,1.000000,3.000000
75%,43.000000,0.000000,15.000000,4.000000,1556.500000,4.000000,3.000000,3.000000,4.000000,8354.500000,4.000000,18.000000,3.000000,4.000000,80.0,1.000000,15.000000,3.000000,3.000000,9.000000,7.000000,3.000000,7.000000
max,60.000000,1.000000,29.000000,5.000000,2065.000000,4.000000,4.000000,5.000000,4.000000,19999.000000,9.000000,25.000000,4.000000,4.000000,80.0,3.000000,40.000000,6.000000,4.000000,37.000000,18.000000,15.000000,17.000000


In [5]:
test.describe()

,Age,DistanceFromHome,Education,EmployeeNumber,EnvironmentSatisfaction,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
count,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.0,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000
mean,36.471429,8.391429,2.868571,1023.285714,2.714286,2.734286,2.068571,2.725714,6479.491429,2.691429,15.200000,3.160000,2.745714,80.0,0.817143,11.202857,2.782857,2.808571,6.782857,4.260000,1.951429,4.017143
std,9.373378,7.685318,1.029583,612.566819,1.067129,0.726669,1.089615,1.083437,4633.609813,2.456892,3.794884,0.367131,1.041226,0.0,0.886539,7.470399,1.295238,0.722488,5.489113,3.622336,2.752532,3.383720
min,18.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1051.000000,0.000000,11.000000,3.000000,1.000000,80.0,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,30.000000,2.000000,2.000000,463.250000,2.000000,2.000000,1.000000,2.000000,2888.750000,1.000000,12.000000,3.000000,2.000000,80.0,0.000000,6.000000,2.000000,2.000000,3.000000,2.000000,0.000000,2.000000
50%,35.000000,7.000000,3.000000,1011.000000,3.000000,3.000000,2.000000,3.000000,5104.000000,2.000000,14.000000,3.000000,3.000000,80.0,1.000000,10.000000,3.000000,3.000000,5.000000,3.000000,1.000000,3.000000
75%,42.000000,11.000000,4.000000,1584.500000,4.000000,3.000000,3.000000,4.000000,8260.250000,4.000000,18.000000,3.000000,4.000000,80.0,1.000000,15.750000,3.000000,3.000000,9.750000,7.000000,2.000000,7.000000
max,60.000000,29.000000,5.000000,2068.000000,4.000000,4.000000,5.000000,4.000000,19973.000000,9.000000,25.000000,4.000000,4.000000,80.0,3.000000,37.000000,6.000000,4.000000,29.000000,16.000000,15.000000,14.000000


In [6]:
train.corr()#系数相关程度

,Age,Attrition,DistanceFromHome,Education,EmployeeNumber,EnvironmentSatisfaction,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
Age,1.000000,-0.175393,0.007081,0.198558,-0.010953,0.011803,0.066528,0.513882,-0.003744,0.500163,0.291211,-0.011259,-0.029613,0.063489,NaN,-0.002413,0.682879,-0.051702,-0.001042,0.328651,0.231842,0.230587,0.212540
Attrition,-0.175393,1.000000,0.088563,-0.046494,-0.045168,-0.097003,-0.122722,-0.168775,-0.125568,-0.155521,0.025889,0.026604,0.046762,-0.051749,NaN,-0.138498,-0.187922,-0.043395,-0.048794,-0.143697,-0.163059,-0.071760,-0.158558
DistanceFromHome,0.007081,0.088563,1.000000,0.011437,0.029930,-0.010308,0.012333,0.016470,-0.009641,-0.017757,-0.016378,0.042627,0.021042,0.018112,NaN,0.050356,0.001287,-0.041208,-0.050950,0.000044,0.019317,-0.002760,0.008852
Education,0.198558,-0.046494,0.011437,1.000000,0.055979,-0.032698,0.022843,0.084075,-0.010201,0.079834,0.118484,-0.008828,-0.000045,-0.006346,NaN,0.035881,0.125672,-0.021629,0.003099,0.074522,0.064363,0.067754,0.071870
EmployeeNumber,-0.010953,-0.045168,0.029930,0.055979,1.000000,0.030428,-0.008589,-0.006121,-0.042443,-0.007147,-0.014460,0.003462,-0.023701,-0.073687,NaN,0.049967,-0.009526,0.017175,0.000548,-0.011106,-0.005649,-0.019621,-0.005333
EnvironmentSatisfaction,0.011803,-0.097003,-0.010308,-0.032698,0.030428,1.000000,-0.028467,-0.015355,0.000212,-0.026410,-0.010743,-0.008882,-0.025044,0.033515,NaN,0.008874,-0.018532,-0.045686,0.026477,-0.012574,0.003572,0.008843,-0.020190
JobInvolvement,0.066528,-0.122722,0.012333,0.022843,-0.008589,-0.028467,1.000000,0.005983,-0.016382,0.006114,0.053557,0.002377,0.000742,0.048363,NaN,0.029483,0.018380,-0.018001,-0.025862,-0.032189,0.001194,-0.031097,0.014176
JobLevel,0.513882,-0.168775,0.016470,0.084075,-0.006121,-0.015355,0.005983,1.000000,-0.005894,0.950776,0.157068,-0.066353,-0.046019,0.042156,NaN,0.002638,0.784020,-0.034620,0.041258,0.544091,0.411481,0.395195,0.376119
JobSatisfaction,-0.003744,-0.125568,-0.009641,-0.010201,-0.042443,0.000212,-0.016382,-0.005894,1.000000,-0.009752,-0.061091,0.019032,-0.011615,-0.033138,NaN,0.021123,-0.023343,0.002754,-0.042767,-0.013772,-0.011798,-0.009761,-0.041852
MonthlyIncome,0.500163,-0.155521,-0.017757,0.079834,-0.007147,-0.026410,0.006114,0.950776,-0.009752,1.000000,0.164627,-0.056469,-0.036086,0.050121,NaN,-0.006320,0.772043,-0.039713,0.032247,0.520641,0.388163,0.386875,0.344237


In [7]:
test['Attrition']=-1
print(len(test.columns))

31


In [8]:
data = train.append(test).reset_index(drop=True)#reset_index(drop=True) 合并列表后删除多余的索引
# data = pd.concat([train,test]).rese_index(drop=True)
data.drop(['Over18', 'StandardHours'], axis=1, inplace=True)
print(data.head())

   Age  Attrition     BusinessTravel              Department  \
0   37          0      Travel_Rarely  Research & Development   
1   54          0  Travel_Frequently  Research & Development   
2   34          1  Travel_Frequently  Research & Development   
3   39          0      Travel_Rarely  Research & Development   
4   28          1  Travel_Frequently  Research & Development   

   DistanceFromHome  Education EducationField  EmployeeNumber  \
0                 1          4  Life Sciences              77   
1                 1          4  Life Sciences            1245   
2                 7          3  Life Sciences             147   
3                 1          1  Life Sciences            1026   
4                 1          3        Medical            1111   

   EnvironmentSatisfaction  Gender  JobInvolvement  JobLevel  \
0                        1    Male               2         2   
1                        4  Female               3         3   
2                        1    Ma

D:\Anaconda3\envs\py36\lib\site-packages\pandas\core\frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [9]:
path='./'
import os
import json
import gc
from numba import jit

#tqdm
from tqdm import tqdm_notebook
from tqdm import tqdm

#Integrated model
import lightgbm as lgb
import catboost as cbt
# import xgboost as xgb

#base import 
import numpy as np
import pandas as pd

# about sklearn
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler as std
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import f1_score

#about time
import time
import datetime 
from datetime import datetime, timedelta

#Garbage collection
import gc

# scipy
from scipy.signal import hilbert
from scipy.signal import hann
from scipy.signal import convolve
from scipy import stats
import scipy.spatial.distance as dist

#other
from collections import Counter 
from statistics import mode 

#warning
import warnings
warnings.filterwarnings("ignore")
import json 
import math
from itertools import product
import ast 

In [10]:
cat_col = [i for i in data.select_dtypes(object).columns if i not in ['Attrition']] # 给除结果列以外的obj型参数编码
for i in tqdm_notebook(cat_col):
    lbl = LabelEncoder()
#     data['count_' + i] = data.groupby([i])[i].transform('count')
    data[i] = lbl.fit_transform(data[i].astype(str))

In [11]:
data.corr()#系数相关程度

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeNumber,EnvironmentSatisfaction,Gender,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,NumCompaniesWorked,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
Age,1.000000,-0.061256,0.025914,-0.034982,-0.007600,0.207929,-0.041778,-0.010412,0.009539,-0.031720,0.032366,0.512555,-0.122534,-0.008441,-0.091467,0.498683,0.299407,0.032344,0.008195,0.006015,0.049485,0.035449,0.678321,-0.020674,-0.017178,0.304106,0.208341,0.209139,0.199053
Attrition,-0.061256,1.000000,0.024336,0.030504,0.088468,-0.002892,-0.031598,-0.018305,-0.042441,-0.011421,-0.059269,-0.084602,0.018132,-0.057359,0.114484,-0.073445,0.011167,0.098584,0.015902,0.014742,-0.041126,-0.076797,-0.088842,-0.013684,-0.054528,-0.056148,-0.082235,-0.004198,-0.065148
BusinessTravel,0.025914,0.024336,1.000000,-0.008741,-0.023478,0.002908,0.021324,-0.013627,-0.005641,-0.032656,0.035302,0.013932,0.005232,-0.037904,0.024238,0.029601,0.023312,0.013774,-0.031893,-0.030361,-0.036594,-0.016959,0.031014,0.017358,-0.014794,-0.019158,-0.013350,-0.037622,-0.025659
Department,-0.034982,0.030504,-0.008741,1.000000,0.020492,0.005288,0.015325,-0.013763,-0.020584,-0.036738,-0.023103,0.100652,0.665253,0.020898,0.057582,0.050735,-0.038215,0.008119,-0.008098,-0.023075,-0.024985,-0.011576,-0.019407,0.038451,0.024370,0.021429,0.054754,0.043992,0.032365
DistanceFromHome,-0.007600,0.088468,-0.023478,0.020492,1.000000,0.019941,0.001144,0.035704,-0.016263,-0.000209,0.009596,0.008210,0.000610,-0.006413,-0.015187,-0.018061,-0.027665,0.027078,0.039921,0.025095,0.004256,0.045451,-0.001994,-0.035472,-0.029434,0.002992,0.018978,-0.001368,0.007865
Education,0.207929,-0.002892,0.002908,0.005288,0.019941,1.000000,-0.033074,0.043131,-0.028633,-0.011553,0.038726,0.096901,0.002161,-0.015419,0.003449,0.089997,0.128999,-0.018644,-0.007768,-0.023793,-0.007607,0.019079,0.144007,-0.021399,0.006430,0.063615,0.056227,0.050087,0.067623
EducationField,-0.041778,-0.031598,0.021324,0.015325,0.001144,-0.033074,1.000000,-0.002636,0.039178,-0.006562,-0.002833,-0.038541,0.017706,-0.033681,0.008377,-0.034083,-0.011667,0.002745,-0.010482,-0.005627,-0.010252,-0.012668,-0.025108,0.042913,0.045054,-0.016303,-0.007259,0.004405,-0.000505
EmployeeNumber,-0.010412,-0.018305,-0.013627,-0.013763,0.035704,0.043131,-0.002636,1.000000,0.012791,0.022119,-0.009312,-0.017176,-0.016052,-0.042865,-0.005551,-0.013722,-0.008234,-0.026145,-0.011960,-0.019557,-0.069329,0.059084,-0.010769,0.023979,0.012998,-0.007046,-0.011791,-0.011243,-0.013419
EnvironmentSatisfaction,0.009539,-0.042441,-0.005641,-0.020584,-0.016263,-0.028633,0.039178,0.012791,1.000000,0.002841,-0.013216,-0.003851,-0.020349,-0.004662,-0.008586,-0.010963,0.011049,0.068673,-0.029471,-0.031335,0.006273,0.004196,-0.007171,-0.018051,0.026045,-0.002558,0.013723,0.011775,-0.011713
Gender,-0.031720,-0.011421,-0.032656,-0.036738,-0.000209,-0.011553,-0.006562,0.022119,0.002841,1.000000,0.020597,-0.036257,-0.036636,0.038299,-0.047421,-0.027705,-0.043097,-0.040490,0.002918,-0.013244,0.023159,0.011876,-0.038942,-0.042726,-0.000797,-0.020520,-0.032852,-0.025262,-0.024698


In [12]:
data.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeNumber,EnvironmentSatisfaction,Gender,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,NumCompaniesWorked,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,37,0,2,1,1,4,1,77,1,1,2,2,4,3,0,5993,1,0,18,3,3,1,7,2,4,7,5,0,7
1,54,0,1,1,1,4,1,1245,4,0,3,3,4,3,0,10502,7,0,17,3,1,1,33,2,1,5,4,1,4
2,34,1,1,1,7,3,1,147,1,1,1,2,2,3,2,6074,1,1,24,4,4,0,9,3,3,9,7,0,6
3,39,0,2,1,1,1,1,1026,4,0,2,4,4,4,1,12742,1,0,16,3,3,1,21,3,3,21,6,11,8
4,28,1,1,1,1,3,3,1111,1,1,2,1,2,2,0,2596,1,0,15,3,1,2,1,2,3,1,0,0,0


In [13]:
feats = [i for i in data.columns if i not in ['Attrition']]
feats

['Age',
 'BusinessTravel',
 'Department',
 'DistanceFromHome',
 'Education',
 'EducationField',
 'EmployeeNumber',
 'EnvironmentSatisfaction',
 'Gender',
 'JobInvolvement',
 'JobLevel',
 'JobRole',
 'JobSatisfaction',
 'MaritalStatus',
 'MonthlyIncome',
 'NumCompaniesWorked',
 'OverTime',
 'PercentSalaryHike',
 'PerformanceRating',
 'RelationshipSatisfaction',
 'StockOptionLevel',
 'TotalWorkingYears',
 'TrainingTimesLastYear',
 'WorkLifeBalance',
 'YearsAtCompany',
 'YearsInCurrentRole',
 'YearsSinceLastPromotion',
 'YearsWithCurrManager']

In [14]:
model = lgb.LGBMClassifier(
        boosting_type="gbdt", num_leaves=50, reg_alpha=0, reg_lambda=0.,
    max_depth=-1, n_estimators=1500, objective='binary',metric= 'F1',
    subsample=0.95, colsample_bytree=0.95, subsample_freq=1,
    learning_rate=0.002, random_state=2017
    )
train_x =data[data['Attrition']!=-1][feats]
train_y =data[data['Attrition']!=-1]['Attrition']
testx= data[data['Attrition']==-1][feats]
# train_x, test_x, train_y, test_y = train_test_split(data1, label1, test_size=0.3, random_state=42)
model.fit(train_x,train_y)
test_pre = model.predict_proba(testx)[:,1]#, num_iteration=model.best_iteration

In [15]:
#5折交叉验证 10折会降低
from sklearn.model_selection import KFold
n_splits=5
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
data.loc[data['Attrition']==2,'Attrition']=0
train_x = data[data['Attrition']!=-1][feats]
train_y = data[data['Attrition']!=-1]['Attrition']
res=data[data['Attrition']==-1][['Attrition']]

test_x= data[data['Attrition']==-1][feats]
res['pred'] = 0
for train_idx, val_idx in kfold.split(train_x):
    model.random_state = model.random_state + 1
    train_x1 = train_x.loc[train_idx]
    train_y1 = train_y.loc[train_idx]
    test_x1 = train_x.loc[val_idx]
    test_y1 = train_y.loc[val_idx]
    #,(vali_x,vali_y)
    model.fit(train_x1, train_y1,eval_set=[(train_x1, train_y1),(test_x1, test_y1)],eval_metric='auc',early_stopping_rounds=100)
    res['pred'] += model.predict_proba(test_x)[:,1]

res['pred'] = res['pred']/5



[1]	training's auc: 0.8737	valid_1's auc: 0.686349
Training until validation scores don't improve for 100 rounds
[2]	training's auc: 0.904998	valid_1's auc: 0.750857
[3]	training's auc: 0.908708	valid_1's auc: 0.746286
[4]	training's auc: 0.913247	valid_1's auc: 0.74146
[5]	training's auc: 0.926996	valid_1's auc: 0.754349
[6]	training's auc: 0.928466	valid_1's auc: 0.742349
[7]	training's auc: 0.929593	valid_1's auc: 0.749587
[8]	training's auc: 0.930534	valid_1's auc: 0.742984
[9]	training's auc: 0.931128	valid_1's auc: 0.746667
[10]	training's auc: 0.931968	valid_1's auc: 0.743746
[11]	training's auc: 0.931294	valid_1's auc: 0.749651
[12]	training's auc: 0.931354	valid_1's auc: 0.75054
[13]	training's auc: 0.932346	valid_1's auc: 0.745841
[14]	training's auc: 0.932044	valid_1's auc: 0.743683
[15]	training's auc: 0.932225	valid_1's auc: 0.750159
[16]	training's auc: 0.932487	valid_1's auc: 0.751175
[17]	training's auc: 0.932104	valid_1's auc: 0.752063
[18]	training's auc: 0.932286	val

[151]	training's auc: 0.95072	valid_1's auc: 0.756952
[152]	training's auc: 0.950881	valid_1's auc: 0.757079
[153]	training's auc: 0.951012	valid_1's auc: 0.757587
[154]	training's auc: 0.950942	valid_1's auc: 0.757968
[155]	training's auc: 0.951153	valid_1's auc: 0.758349
[156]	training's auc: 0.951052	valid_1's auc: 0.758095
[157]	training's auc: 0.951052	valid_1's auc: 0.758476
[158]	training's auc: 0.951123	valid_1's auc: 0.758095
[159]	training's auc: 0.951264	valid_1's auc: 0.758984
[160]	training's auc: 0.951485	valid_1's auc: 0.758349
[161]	training's auc: 0.951626	valid_1's auc: 0.758095
[162]	training's auc: 0.951686	valid_1's auc: 0.758476
[163]	training's auc: 0.951777	valid_1's auc: 0.757968
[164]	training's auc: 0.952049	valid_1's auc: 0.757587
[165]	training's auc: 0.95221	valid_1's auc: 0.757714
[166]	training's auc: 0.952139	valid_1's auc: 0.757079
[167]	training's auc: 0.952451	valid_1's auc: 0.757079
[168]	training's auc: 0.952391	valid_1's auc: 0.756825
[169]	traini

[39]	training's auc: 0.923423	valid_1's auc: 0.842995
[40]	training's auc: 0.923795	valid_1's auc: 0.843599
[41]	training's auc: 0.923776	valid_1's auc: 0.843599
[42]	training's auc: 0.923556	valid_1's auc: 0.843599
[43]	training's auc: 0.923451	valid_1's auc: 0.844354
[44]	training's auc: 0.923213	valid_1's auc: 0.845411
[45]	training's auc: 0.924682	valid_1's auc: 0.845109
[46]	training's auc: 0.924434	valid_1's auc: 0.845411
[47]	training's auc: 0.924787	valid_1's auc: 0.846769
[48]	training's auc: 0.924692	valid_1's auc: 0.847222
[49]	training's auc: 0.925684	valid_1's auc: 0.846467
[50]	training's auc: 0.926056	valid_1's auc: 0.847373
[51]	training's auc: 0.926553	valid_1's auc: 0.847222
[52]	training's auc: 0.926705	valid_1's auc: 0.84843
[53]	training's auc: 0.926524	valid_1's auc: 0.848128
[54]	training's auc: 0.926381	valid_1's auc: 0.848128
[55]	training's auc: 0.926314	valid_1's auc: 0.847826
[56]	training's auc: 0.927392	valid_1's auc: 0.847675
[57]	training's auc: 0.92785	

[189]	training's auc: 0.946945	valid_1's auc: 0.857337
[190]	training's auc: 0.946992	valid_1's auc: 0.857941
[191]	training's auc: 0.947212	valid_1's auc: 0.85779
[192]	training's auc: 0.947288	valid_1's auc: 0.857488
[193]	training's auc: 0.947288	valid_1's auc: 0.857186
[194]	training's auc: 0.947374	valid_1's auc: 0.857337
[195]	training's auc: 0.947536	valid_1's auc: 0.857941
[196]	training's auc: 0.947555	valid_1's auc: 0.85779
[197]	training's auc: 0.947622	valid_1's auc: 0.857337
[198]	training's auc: 0.947927	valid_1's auc: 0.857035
[199]	training's auc: 0.948109	valid_1's auc: 0.857186
[200]	training's auc: 0.948385	valid_1's auc: 0.856733
[201]	training's auc: 0.948586	valid_1's auc: 0.855978
[202]	training's auc: 0.948624	valid_1's auc: 0.855525
[203]	training's auc: 0.948672	valid_1's auc: 0.855072
[204]	training's auc: 0.948824	valid_1's auc: 0.855374
[205]	training's auc: 0.94892	valid_1's auc: 0.85628
[206]	training's auc: 0.94912	valid_1's auc: 0.855374
[207]	training'

[53]	training's auc: 0.926052	valid_1's auc: 0.738095
[54]	training's auc: 0.926107	valid_1's auc: 0.738095
[55]	training's auc: 0.926404	valid_1's auc: 0.738095
[56]	training's auc: 0.926442	valid_1's auc: 0.737413
[57]	training's auc: 0.926367	valid_1's auc: 0.7369
[58]	training's auc: 0.926469	valid_1's auc: 0.737925
[59]	training's auc: 0.926544	valid_1's auc: 0.739631
[60]	training's auc: 0.926432	valid_1's auc: 0.738778
[61]	training's auc: 0.926516	valid_1's auc: 0.738778
[62]	training's auc: 0.926293	valid_1's auc: 0.738095
[63]	training's auc: 0.927068	valid_1's auc: 0.739205
[64]	training's auc: 0.927207	valid_1's auc: 0.739205
[65]	training's auc: 0.927049	valid_1's auc: 0.739375
[66]	training's auc: 0.92743	valid_1's auc: 0.738693
[67]	training's auc: 0.927959	valid_1's auc: 0.740399
[68]	training's auc: 0.928005	valid_1's auc: 0.741082
[69]	training's auc: 0.927959	valid_1's auc: 0.740911
[70]	training's auc: 0.928209	valid_1's auc: 0.739546
[71]	training's auc: 0.928525	v

[98]	training's auc: 0.933508	valid_1's auc: 0.744912
[99]	training's auc: 0.933711	valid_1's auc: 0.745263
[100]	training's auc: 0.933572	valid_1's auc: 0.746842
[101]	training's auc: 0.933942	valid_1's auc: 0.746491
[102]	training's auc: 0.934154	valid_1's auc: 0.746316
[103]	training's auc: 0.934403	valid_1's auc: 0.745789
[104]	training's auc: 0.934588	valid_1's auc: 0.744912
[105]	training's auc: 0.934394	valid_1's auc: 0.744737
[106]	training's auc: 0.934145	valid_1's auc: 0.744035
[107]	training's auc: 0.934043	valid_1's auc: 0.744035
Early stopping, best iteration is:
[7]	training's auc: 0.924079	valid_1's auc: 0.75614
[1]	training's auc: 0.870038	valid_1's auc: 0.676102
Training until validation scores don't improve for 100 rounds
[2]	training's auc: 0.908164	valid_1's auc: 0.70305
[3]	training's auc: 0.92285	valid_1's auc: 0.737923
[4]	training's auc: 0.925059	valid_1's auc: 0.777325
[5]	training's auc: 0.930718	valid_1's auc: 0.771211
[6]	training's auc: 0.931701	valid_1's a

In [16]:
pd.DataFrame({'result':res['pred']}).to_csv(path+'output/lgb_proba_leave50.csv',index=False)

In [17]:
from sklearn.ensemble import GradientBoostingClassifier
model=GradientBoostingClassifier()
testX=test_x.values
trainY=train_y.values

trainX=train_x.values
model.fit(trainX,trainY)
predictY=model.predict_proba(testX)[:,1]

In [18]:
pd.DataFrame({'result':predictY}).to_csv(path+'output/gbdt_proba.csv',index=False)

In [19]:
res.loc[res['pred']>=0.5,'pred']=1
res.loc[res['pred']<0.5,'pred']=0
res.columns=['Attrition','result']
res['result']=res['result'].astype(int)
res[['result']].to_csv(path+'output/lgb_submission_5_fold_leave50.csv', index=False)